In [1]:
!pip install tensorflow

from IPython.display import clear_output
from google.colab import drive
import os.path
from os import path
import collections
import pathlib
import tensorflow as tf
from tensorflow.keras import utils
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

drive.mount('/content/gdrive')
gdrive_path = '/content/gdrive/My Drive/recommender_dataset'

if not path.exists(gdrive_path):
  os.mkdir(gdrive_path)

os.chdir(gdrive_path)
%cd '/content/gdrive/MyDrive/recommender_dataset/'

# Set the destination path
destination_path='/content/gdrive/MyDrive/recommender_dataset/'

# Create the destination directory if it doesn't exist
!mkdir -p "$destination_path"

# Download the tar.gz file
!wget -O "$destination_path/criteo-research-kaggle-display-advertising-challenge-dataset.tar.gz" "https://go.criteo.net/criteo-research-kaggle-display-advertising-challenge-dataset.tar.gz"

# Extract the tar.gz file
!tar -xzvf "$destination_path/criteo-research-kaggle-display-advertising-challenge-dataset.tar.gz" -C "$destination_path"

# Remove the downloaded tar.gz file if desired
!rm "$destination_path/criteo-research-kaggle-display-advertising-challenge-dataset.tar.gz"

clear_output()

In [27]:
destination_path='/content/gdrive/MyDrive/recommender_dataset/'
file = destination_path + 'train.txt'
print(file)
columns = ['label', *(f'I{i}' for i in range(1, 14)), *(f'C{i}' for i in range(1, 27))]
df = pd.read_csv(file, nrows=1000000, sep='\t', names=columns)

/content/gdrive/MyDrive/recommender_dataset/train.txt


Preprocessing

In [28]:
cat_columns = [f'C{i}' for i in range(1, 27)]
int_columns = [f'I{i}' for i in range(1, 14)]

In [4]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X = df.fillna(0)
labels = X['label']
integer_features = X.iloc[:, 1:14]  # I1-I13
categorical_features = X.iloc[:, 14:]  # C1-C26

# Convert categorical features to numerical representations (hashing)
label_encoders = {}
for col in cat_columns:
    le = preprocessing.LabelEncoder()
    categorical_features[col] = le.fit_transform(categorical_features[col].astype(str))
    label_encoders[col] = le

features = pd.concat([integer_features, categorical_features], axis=1)

# Split dataset into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Prepare target and features
train_numerical_features = train_data[['I' + str(i) for i in range(1, 14)]]
train_categorical_features = train_data[cat_columns]

test_numerical_features = test_data[['I' + str(i) for i in range(1, 14)]]
test_categorical_features = test_data[cat_columns]

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten
from tensorflow import keras
from tensorflow.keras.models import Model

def build_mlp(input_dim):
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_dim=input_dim),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
embedding_dim = 8
num_cat_cols = len(cat_columns)
num_int_cols = len(int_columns)

input_dim = num_cat_cols + num_int_cols

mlp_model = build_mlp(input_dim)
mlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

mlp_model.fit(
    x=tf.concat([train_categorical_features, train_numerical_features], axis=1),
    y=train_labels,
    epochs=10,
    batch_size=128,
    validation_split=0.1
)
mlp_model.save('/content/gdrive/MyDrive/recommender_res/q1/mlp_model.h5')

Epoch 1/10
5625/5625 [==============================] - 54s 9ms/step - loss: 157.6760 - auc: 0.5244 - val_loss: 108.4418 - val_auc: 0.5023
Epoch 2/10
5625/5625 [==============================] - 56s 10ms/step - loss: 52.4718 - auc: 0.5280 - val_loss: 91.5621 - val_auc: 0.5064
Epoch 3/10
5625/5625 [==============================] - 31s 6ms/step - loss: 18.0238 - auc: 0.5324 - val_loss: 7.0044 - val_auc: 0.5350
Epoch 4/10
5625/5625 [==============================] - 19s 3ms/step - loss: 5.7036 - auc: 0.5315 - val_loss: 1.2329 - val_auc: 0.5270
Epoch 5/10
5625/5625 [==============================] - 20s 4ms/step - loss: 0.8384 - auc: 0.5319 - val_loss: 0.5702 - val_auc: 0.5839
Epoch 6/10
5625/5625 [==============================] - 19s 3ms/step - loss: 0.5684 - auc: 0.5148 - val_loss: 0.5686 - val_auc: 0.5002
Epoch 7/10
5625/5625 [==============================] - 19s 3ms/step - loss: 0.5708 - auc: 0.4979 - val_loss: 0.5686 - val_auc: 0.5000
Epoch 8/10
5625/5625 [=========================

In [ ]:
# Copy the dataframe
eval_data_mlp = test_data.copy()

# Drop the column "col2" from the copied dataframe
mlp_auc, mlp_loss = mlp_model.evaluate(eval_data_mlp, test_labels)
print("MLP AUC:", mlp_auc, "Loss:", mlp_loss)

6250/6250 [==============================] - 36s 6ms/step - loss: 803.6530 - auc: 0.4881
MLP AUC: 803.6530151367188 Loss: 0.4881417751312256


In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Layer

class FactorizationMachine(Layer):
    def __init__(self, input_dim, num_factors, **kwargs):
        super(FactorizationMachine, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.num_factors = num_factors

    def build(self, input_shape):
        self.V = self.add_weight(name='V',
                                 shape=(self.input_dim, self.num_factors),
                                 initializer='uniform',
                                 trainable=True)
        super(FactorizationMachine, self).build(input_shape)

    def call(self, inputs):
        linear_terms = tf.reduce_sum(inputs, axis=1, keepdims=True)
        interactions = 0.5 * tf.reduce_sum(
            tf.square(tf.matmul(inputs, self.V) -
                      tf.matmul(tf.square(inputs), tf.square(self.V))),
            axis=1, keepdims=True)
        output = linear_terms + interactions
        return output

def build_mlp_fm(input_dim, fm_num_factors):
    input_layer = keras.layers.Input(shape=(input_dim,))

    # MLP layers
    mlp = keras.Sequential([
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(64, activation='relu')
    ])
    mlp_output = mlp(input_layer)

    # FM layer
    fm_output = FactorizationMachine(input_dim=input_dim, num_factors=fm_num_factors)(input_layer)

    # Concatenate MLP and FM outputs
    concatenated = keras.layers.Concatenate()([mlp_output, fm_output])

    # Final MLP layer
    final_output = keras.layers.Dense(1, activation='sigmoid')(concatenated)

    model = keras.Model(inputs=input_layer, outputs=final_output)
    return model

fm_num_factors = 10  # Number of factors for FM

mlp_fm_model = build_mlp_fm(input_dim, fm_num_factors)
mlp_fm_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 39)]         0           []                               
                                                                                                  
 sequential (Sequential)        (None, 64)           13376       ['input_1[0][0]']                
                                                                                                  
 factorization_machine (Factori  (None, 1)           390         ['input_1[0][0]']                
 zationMachine)                                                                                   
                                                                                                  
 concatenate_2 (Concatenate)    (None, 65)           0           ['sequential[0][0]',       

In [ ]:
mlp_fm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

mlp_fm_model.fit(
    x=tf.concat([train_categorical_features, train_numerical_features], axis=1),
    y=train_labels,
    epochs=10,
    batch_size=128,
    validation_split=0.1
)
mlp_fm_model.save('/content/gdrive/MyDrive/recommender_res/q2/mlp_fm_model.h5')

Epoch 1/10
5625/5625 [==============================] - 24s 4ms/step - loss: 18935346888704.0000 - auc: 0.5045 - val_loss: 361718752.0000 - val_auc: 0.5135
Epoch 2/10
5625/5625 [==============================] - 39s 7ms/step - loss: 70213312.0000 - auc: 0.5208 - val_loss: 987622.6875 - val_auc: 0.5294
Epoch 3/10
5625/5625 [==============================] - 24s 4ms/step - loss: 216034.5469 - auc: 0.5230 - val_loss: 499648.6250 - val_auc: 0.5047
Epoch 4/10
5625/5625 [==============================] - 22s 4ms/step - loss: 765967616.0000 - auc: 0.5241 - val_loss: 69775504.0000 - val_auc: 0.5160
Epoch 5/10
5625/5625 [==============================] - 37s 7ms/step - loss: 10482634.0000 - auc: 0.5288 - val_loss: 12025151.0000 - val_auc: 0.5849
Epoch 6/10
5625/5625 [==============================] - 51s 9ms/step - loss: 1038822592.0000 - auc: 0.5273 - val_loss: 2375834.5000 - val_auc: 0.5566
Epoch 7/10
5625/5625 [==============================] - 49s 9ms/step - loss: 2008970624.0000 - auc: 0.5

In [ ]:
# Copy the dataframe
eval_data_mlp_fm = test_data.copy()

# Drop the column "col2" from the copied dataframe
mlp_fm_auc, mlp_fm_loss = mlp_fm_model.evaluate(eval_data_mlp_fm, test_labels)
print("MLP+FM AUC:", mlp_fm_auc, "MLP+FM Loss:", mlp_fm_loss)

6250/6250 [==============================] - 40s 6ms/step - loss: 31842531328.0000 - auc: 0.5000
MLP+FM AUC: 31842531328.0 MLP+FM Loss: 0.5


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Reshape

def build_fint_model(num_fields, num_categories_per_field, embedding_dim, hidden_units):
    inputs = []
    embedding_layers = []

    for i in range(num_fields):
        input_layer = Input(shape=(1,), name=f'field_{i}')
        embedding_layer = Embedding(input_dim=num_categories_per_field[i], output_dim=embedding_dim)(input_layer)
        inputs.append(input_layer)
        embedding_layers.append(embedding_layer)

    # Interaction layers for categorical fields
    interactions = []
    for i in range(num_fields):
        for j in range(i + 1, num_fields):
            interactions.append(embedding_layers[i] * embedding_layers[j])


    field_interactions = Concatenate()(interactions)
    field_interactions = Flatten()(field_interactions)

    # Concatenate embedding outputs with numeric inputs
    combined = Concatenate()([Flatten()(embedding_layer) for embedding_layer in embedding_layers] + [field_interactions])


    for units in hidden_units:
        combined = Dense(units, activation='relu')(combined)

    output_layer = Dense(1, activation='sigmoid')(combined)

    model = tf.keras.Model(inputs=inputs, outputs=output_layer)
    return model


embedding_dim = 8
hidden_units = [64, 32]
num_cat_per_col = [len(train_data[feature].unique()) for feature in cat_columns]
fint_model = build_fint_model(len(num_cat_per_col), num_cat_per_col, embedding_dim, hidden_units)
fint_model.summary()


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Reshape
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder



# Model Architecture
def build_finn_model(num_numeric_features, num_categorical_features, num_categories_per_field, embedding_dim, hidden_units):
    # Numeric input
    numeric_input = Input(shape=(num_numeric_features,), name='numeric_input')

    # Categorical inputs
    categorical_inputs = []
    embedding_layers = []

    for i in range(num_categorical_features):
        input_layer = Input(shape=(1,), name=f'categorical_input_{i}')
        embedding_layer = Embedding(input_dim=num_categories_per_field[i], output_dim=embedding_dim)(input_layer)
        categorical_inputs.append(input_layer)
        embedding_layers.append(embedding_layer)

    # Field-aware interactions
    interactions = []
    for i in range(num_categorical_features):
        for j in range(i + 1, num_categorical_features):
            interactions.append(Flatten()(embedding_layers[i]) * Flatten()(embedding_layers[j]))

    field_interactions = Concatenate()(interactions)
    field_interactions = Flatten()(field_interactions)


    # Combine inputs
    combined = Concatenate()([numeric_input, Flatten()(field_interactions)])

    # Fully connected layers
    for units in hidden_units:
        combined = Dense(units, activation='relu')(combined)

    # Output layer
    output_layer = Dense(1, activation='sigmoid')(combined)

    model = tf.keras.Model(inputs=[numeric_input] + categorical_inputs, outputs=output_layer)
    return model

# Hyperparameters
num_numeric_features = 13
num_categorical_features = 26
num_cat_per_col = [len(train_data[feature].unique()) for feature in cat_columns]
embedding_dim = 8
hidden_units = [64, 32]

# Build the model
finn_model = build_finn_model(num_numeric_features, num_categorical_features, num_cat_per_col, embedding_dim, hidden_units)

# Compile the model
finn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])


In [ ]:
finn_model.summary()

In [6]:
# Train the model
finn_model.fit(
    x=[train_numerical_features] + [train_data[col].values for col in cat_columns],
    y=train_labels,
    epochs=10,
    batch_size=128,
    validation_split=0.1
)
finn_model.save('/content/gdrive/MyDrive/recommender_res/q3/finn_model.h5')

Epoch 1/10
5625/5625 [==============================] - 397s 64ms/step - loss: 1.7615 - auc: 0.6913 - val_loss: 0.8776 - val_auc: 0.7309
Epoch 2/10
5625/5625 [==============================] - 192s 34ms/step - loss: 0.4873 - auc: 0.7998 - val_loss: 0.5105 - val_auc: 0.7241
Epoch 3/10
5625/5625 [==============================] - 190s 34ms/step - loss: 0.3861 - auc: 0.8607 - val_loss: 0.5141 - val_auc: 0.7349
Epoch 4/10
5625/5625 [==============================] - 180s 32ms/step - loss: 0.3548 - auc: 0.8839 - val_loss: 0.5147 - val_auc: 0.7487
Epoch 5/10
5625/5625 [==============================] - 211s 37ms/step - loss: 0.3298 - auc: 0.8986 - val_loss: 0.5363 - val_auc: 0.7491
Epoch 6/10
5625/5625 [==============================] - 202s 36ms/step - loss: 0.3140 - auc: 0.9085 - val_loss: 0.5454 - val_auc: 0.7366
Epoch 7/10
5625/5625 [==============================] - 180s 32ms/step - loss: 0.3014 - auc: 0.9157 - val_loss: 0.5440 - val_auc: 0.7418
Epoch 8/10
5625/5625 [===================

In [10]:
# Copy the dataframe
eval_data_mlp_fm = test_data.copy()

# Drop the column "col2" from the copied dataframe
fint_auc, fint_loss = finn_model.evaluate([test_numerical_features] + [eval_data_mlp_fm[col].values for col in cat_columns], test_labels)
print("FINT AUC:", fint_auc, "FINT Loss:", fint_loss)

6250/6250 [==============================] - 77s 12ms/step - loss: 0.6206 - auc: 0.7294
FINT AUC: 0.6205617189407349 FINT Loss: 0.7293546199798584



**Models** | MLP | MLP+FM | FINT
---    | --- |   ---    | ---
**AUC**    | `0.48`|`0.5` |`0.72`


In [13]:
def recommend_top_items(model, user_features, N):
    predicted_scores = model.predict(user_features)
    top_indices = predicted_scores.argsort()[::-1][:N]
    return top_indices


Q3.2. Recommend any given user to the 5 most relevant items for all implemented
models.

In [33]:
from tensorflow import keras

# Define a function to recommend top N items for a given user
def recommend_top_items(model, user_features, N):
    predicted_scores = model.predict(user_features)
    top_indices = predicted_scores.argsort()[::-1][:N]
    return top_indices

top_N_recommendations = 5

mlp_model = keras.models.load_model('/content/gdrive/MyDrive/recommender_res/q1/mlp_model.h5')
mlp_recommendations = recommend_top_items(mlp_model, test_data, top_N_recommendations)

mlp_fm_model = keras.models.load_model('/content/gdrive/MyDrive/recommender_res/q2/mlp_fm_model.h5', custom_objects={"FactorizationMachine": FactorizationMachine})
mlp_fm_recommendations = recommend_top_items(mlp_fm_model, test_data, top_N_recommendations)

fint_model = keras.models.load_model('/content/gdrive/MyDrive/recommender_res/q3/finn_model.h5')
fint_recommendations = recommend_top_items(fint_model, [test_numerical_features] + [test_data[col].values for col in cat_columns], top_N_recommendations)

print("MLP Recommendations:", mlp_recommendations)
print("MLP+FM Recommendations:", mlp_fm_recommendations)
print("FINT Recommendations:", fint_recommendations)

6250/6250 [==============================] - 62s 10ms/step
MLP Recommendations: [[0]
 [0]
 [0]
 [0]
 [0]]
MLP+FM Recommendations: [[0]
 [0]
 [0]
 [0]
 [0]]
FINT Recommendations: [[0]
 [0]
 [0]
 [0]
 [0]]


**Models** | MLP | MLP+FM | FINT
---    | --- |   ---    | ---
**AUC**    | `0.48`|`0.5` |`0.72`
